In [1]:
import time
import random

<h1>TP: Dynamic Programming</h1>

<h2>1. Introduction: Packet Scheduling</h2>

Consider a router handling packets from flows of $K$ different priority classes.<br>
At a given moment, there is a fixed number $N_k$ of packets for each class $k=1,\dots,K$. <br>
All packets have the same length $L$ [bits], but packets of priority class $k$ have transmission rate of $R_k$ [bps], such that $R_1 > R_2 > \dots > R_K$. <br>
Therefore, a packet of class $k$ takes $T_k=L / R_k$ [s] to be transmitted.

Packets are organized in a queue $Q_t$ that changes at every iteration $t$, depending on which packet was chosen to be transmitted, i.e., the scheduling policy. <br>
At every iteration $t$, the router selects a packet of a given class $k$ to be transmitted, which takes $T_k$ to finish the transmission. <br>
This means that the remaining packets in the queue must wait $T_k$ before having another chance to be transmitted. <br>
We define the worst-case (queue) wait time of all packets in the queue $Q_t$ at iteration $t$ as: <br>

$W_t = \sum_{p\in Q_t} T_{k(p)}$, where $k(p)$ is the class of packet $p$ <br>


The goal of this activity is to find a <u><b>scheduling policy</b></u> $\pi^*$ that is able to <u><b>minimize the total worst-case wait time over all iterations until all packets are transmitted</u></b>, i.e.,<br>

$\min \sum_{t=1}^{\infty} W_t = \sum_{t=1}^{\infty} \sum_{p\in Q_t} T_{k(p)}$.

We call it the <i>Packet Scheduling Problem</i>.

<h2> 2. MDP for Packet Scheduling</h2>

<h3> 2.1. Mathematical Model </h3>

The first step is to represent the problem as an MDP with the following components: <br>
- Set of states: $\mathcal S=\{\mathbf{s}=(n_1,n_2,\dots,n_K)\}_{n_i=1,\dots,N_i, i=1,\dots,K}$ <br>
- Set of actions $\mathcal A=\{k\}_{k=1,\dots,K}$ <br>
- Discount Factor: $\gamma$
- The MDP has an initial state $\mathbf{s}_0=(N_1,N_2,\dots,N_K)$ and a terminal (absorbing) state $\mathbf{s}'=(0,0,\dots, 0)$

The action of transmitting a packet from class $k$ is represented by the following transformation: <br>

$T_k: \mathcal S \rightarrow \mathcal S$, such that $T_k(\mathbf{s}) = \mathbf{s} \ominus \mathbf{e}_k$,<br>

where <br>
- $\mathbf{e}_k \in \{0,1\}^{1\times K}$ is a vector whose entry $k$ is $1$ and all other entries are $0$, and<br>
- $\ominus$ operator is an adapted vectorial subtraction operator, where any negative outcome is converted to zero.

For example, we have $K=2$ classes and $N_1=N_2=3$ packets for each class. From the initial state $\mathbf{s}_0=(N_1,N_2) = (3,3)$ , if we choose to transmit a packet from class $1$, we apply the transformation $T_1(\cdot)$ as follows, <br>

$T_1(\mathbf{s}_0= (3,3)) = \mathbf{s}_0 \ominus \mathbf{e}_1 = (3,3) \ominus (1,0) = (2,3)$. <br>

Therefore, if we are in state $\mathbf{s}_0=(N_1,N_2) = (3,3)$ and transmit a packet from class $1$, then we land at state $\mathbf{s}'=(2,3)$.<br>
On the other hand, if we are in state $s=(3,0)$ and decide to transmit a packet from class $2$, then $T_2(\mathbf{s}= (3,0)) = (3,0) \ominus (0,1) = (3,0)$, i.e., no changes.


In this activity, all transmissions (actions) lead deterministically to the corresponding immediate smaller state, we have that the transition probabilities are:<br> 

$\mathcal P_{s,s'}^{a=T_k} = \begin{cases} 1, & \text{ if } T_k(s)=s' \\ 0, & \text{otherwise}\end{cases}$,  $\forall s,s'\in\mathcal{S}, \forall a\in\mathcal{A}$<br>

<b><u>Exercise</u></b>: Draw the MDP for the Packet Scheduling Problem for $K=2$ classes of priority.

<h3> 2.2. Computational Model </h3>

The code below implements an MDP Python class for $K=2$ priority classes.

In [2]:
class MDP:
    def __init__(self, classes=[3, 3]):        
        self.K = 2              # Number of classes
        self.N = classes        # Number of packets of each class

        # set of states
        self.S = [(i,j) for i in range(self.N[0],-1,-1) for j in range(self.N[1],-1,-1)]
        # self.S = [(i,j) for i in range(self.N[0]+1) for j in range(self.N[1]+1)]

        # set of actions: Tx1 = transmits packet of class 1; Tx2 = transmits packet of class 2
        self.A  = ["Tx1", "Tx2"]                                            

        # definition of transition probabilities
        self.P = { (s, ss, a): 1 if self.transmit(s,a) == ss else 0 for s in self.S for ss in self.S for a in self.A}  

        # definition of transmission delay of each class
        self.packet_size = 1500*8                                               # 1500 Bytes
        self.data_rate = [100e6, 10e6]                                          # Rate 1: 100Mbps, Rate 2: 10Mbps
        self.T = [self.packet_size / rate * 10**3 for rate in self.data_rate]   # Transmission delay [ms]

        # generate queue with N1 and N2 packets randomly arranged
        self.Q = random.sample([1] * self.N[0] + [2] * self.N[1], self.N[0] + self.N[1])
        
    # support function "transmission" that help define the transition probabilities Tp and rewards R
    def transmit(self, s, a):
        if s == (0,0):
            return s
        if a == "Tx1":
            return (max(0, s[0]-1), s[1])
        if a == "Tx2":
            return (s[0], max(0, s[1]-1))

<b><u>Exercise</u></b>: <br>
- The set of rewards $\{\mathcal{R}_{s}^a\}_{a\in\mathcal{A}, s\in\mathcal{S}}$ is still missing from the MDP definition in Section 2.1. How would you choose the reward for this activity? Recall that there must be a reward for every action you take at every state. Explain your reasoning and add it the computational model in the code below.<br>
- What about the discount factor $\gamma$? Is there an appropriate value? Explain your reasoning and add it the computational model in the code below.

In [3]:
# Instance of MDP object
mdp = MDP()
print(mdp.Q)

# Rewards
# TODO: add the definition of reward
mdp.R = {}

# Discount factor, gamma
# TODO: Exercise 3: add value for the discount factor
mdp.g = 0.0        

[2, 1, 1, 1, 2, 2]


<h2> 3. Testing Scheduling Policies </h2>

Now, we will evaluate the performance of different policies.

<h3> 3.1. Policy Evaluation </h3>

The method we will use is the Policy Evaluation (PE), which relies on the iterative application of Bellman Expectation Equation: <br>

$v_t(s) = \sum_{a\in\mathcal{A}} \pi(a|\,s) \left[\mathcal{R}_s^a + \gamma \sum_{s'\in\mathcal{S}} \mathcal{P}_{ss'}^a\cdot v_{t-1}(s')\right]$

<b><u>Exercise</b></u>: Use the Bellman Expectation Equation to finish the implementation of the PE function below:

In [4]:
# Policy Evaluation Algorithm
# Input:  MDP parameters (states S, actions A, rewards R, transition probabilities Tp)
#         policy Pi
#         initial value v0
#         maximum number of iterations max_iter
# Output: list of the max_iter values for the states (last item of the list estimates v_pi)
def policy_eval(mdp, Pi, v0, max_iter=100):
    ## TODO: implement PE
    return

<h3> 3.2. RANDOM Policy </h3>

The first policy we will try is the RANDOM policy, which selects the next action uniformly at random from the set of available actions at every state.

The implementation of a RANDOM policy is given below:

In [5]:
Pi_RANDOM = { (s, a): 1.0/len(mdp.A) for s in mdp.S for a in mdp.A }

# evaluate policy Pi over max_iter iterations and store value functions of every iteration
v_RANDOM = policy_eval(mdp, Pi_RANDOM, v0={s: 0 for s in mdp.S}, max_iter=1000)[-1]

## print results
print("These are the values of the random policy")
for s in mdp.S:
    print(" %.5f" %v_RANDOM[s] if v_RANDOM[s]==0 else "%.5f" %v_RANDOM[s], end='\n' if s[1] == 0 else '\t')

These are the values of the random policy
-5.19750	-4.53750	-4.12500	-3.96000
-4.53750	-3.63000	-2.97000	-2.64000
-4.12500	-2.97000	-1.98000	-1.32000
-3.96000	-2.64000	-1.32000	 0.00000


<h3> 3.3. FIFO Policy </h3>

In First-In, First-Out (FIFO) scheduling discipline, we process the packets in order they are organized in the queue.

In [6]:
Pi_FIFO = {}
for s in mdp.S:
    if s == (0,0):
        Pi_FIFO[(s, "Tx1")] = 1.0
        Pi_FIFO[(s, "Tx2")] = 0.0
    else:
        for a in mdp.A:
            Pi_FIFO[(s, a)] = 1.0  if "Tx%d" %mdp.Q[mdp.N[0] + mdp.N[1] - s[0] - s[1]] == a else 0.0

Now, we evaluate the FIFO policy and are able to compare it with the RANDOM one.

In [7]:
# evaluate policy Pi over max_iter iterations and store value functions of every iteration
v_FIFO = policy_eval(mdp, Pi_FIFO, v0={s: 0 for s in mdp.S}, max_iter=1000)[-1]

## print results
print("These are the values of the FIFO policy")
for s in mdp.S:
    print(" %.5f" %v_FIFO[s] if v_FIFO[s]==0 else "%.5f" %v_FIFO[s], end='\n' if s[1] == 0 else '\t')

These are the values of the FIFO policy
-3.96000	-2.76000	-1197.84000	-1198.92000
-120.00000	-2.64000	-1198.92000	-1200.00000
-120.00000	-2.52000	-1200.00000	-1200.00000
-120.00000	-2.40000	-1.20000	 0.00000


<h2> 5. Finding the Optimal Scheduling Policy </h2>



<h3> 5.1. Value Iteration</h3>

The method we will use is called the Value Iteration (VI), which relies on the iterative application of Bellman Optimality Equation: <br>

$v_t(s) = \max_{a\in\mathcal{A}} \left[\mathcal{R}_s^a + \gamma \sum_{s'\in\mathcal{S}} \mathcal{P}_{ss'}^a\cdot v_{t-1}(s')\right]$

<b><u>Exercise:</b></u> Use the Bellman Optimality Equation to finish the implementation of the VI function below:

In [8]:
# Value Iteration Algorithm
# Input:    MDP parameters (states S, actions A, etc.)
# Output:   (Estimate of the) Optimal values
#           Optimal policy
def value_iteration(mdp, max_iter=100):
    # TODO: Implement VI
    return

<h3> 5.2. Testing Value Iteration </h3>

In [9]:
# print optimal value obtained by Policy Iteration
print("These are the values of the FIFO policy")
for s in mdp.S:
    print(" %.5f" %v_FIFO[s] if v_FIFO[s]==0 else "%.5f" %v_FIFO[s], end='\n' if s[1] == 0 else '\t')
    
# compute optimal value for the MDP
v, Pi_OPT = value_iteration(mdp, max_iter=1000)
v_OPT = v[-1]

# print optimal value obtained by Value Iteration
print("\nThese are the values of the Greedy policy")
for s in mdp.S:
    print(" %.5f" %v_OPT[s] if v_OPT[s]==0 else "%.5f" %v_OPT[s], end='\n' if s[1] == 0 else '\t')

These are the values of the FIFO policy
-3.96000	-2.76000	-1197.84000	-1198.92000
-120.00000	-2.64000	-1198.92000	-1200.00000
-120.00000	-2.52000	-1200.00000	-1200.00000
-120.00000	-2.40000	-1.20000	 0.00000

These are the values of the Greedy policy
-3.96000	-2.76000	-1.56000	-0.36000
-3.84000	-2.64000	-1.44000	-0.24000
-3.72000	-2.52000	-1.32000	-0.12000
-3.60000	-2.40000	-1.20000	 0.00000


<b><u>Exercise:</b></u> Using the results obtained from the application of value iteration, answer: What is the optimal policy for the Scheduling Problem?